## test to generate S2 VI time series data <br>

we use the ts_datacube_extraction function but modify the collection data that we only get the Sentinel-2 data cube. <br>
On that Sentinel-2 timeseries cube we will apply the Awsome-package to add the VI's to the S2 data cube.

All data is scaled to Int16 with nodata value set to -32768! The data is extracted for 1 year and temporal resampled to 10-daily intervals -> 36 time stamps per year.

In [1]:
import openeo
from extentmapping.openeo.preprocessing import ts_datacube_extraction
from extentmapping.config import get_job_options, get_collection_options, get_advanced_dataextraction_options
from extentmapping.utils import laea20km_id_to_extent, reproj_bbox_to_ll
from openeo.extra.spectral_indices import compute_and_rescale_indices
import os
import json

C:\Users\BUCHHORM\Anaconda3\envs\extent\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


### connect to the backend

In [2]:
#backend = 'terrascope'
backend = 'development'    # for current tests

# Make connection
if backend == 'terrascope':
    connection = openeo.connect("https://openeo.vito.be").authenticate_oidc()
elif backend == 'development':
    connection = openeo.connect("https://openeo-dev.vito.be").authenticate_oidc()
else:
    print('currently no specific connections to backends like creodias and sentinelhub are setup.')
    print('use standard entry point')
    connection = openeo.connect("https://openeo.cloud").authenticate_oidc()

Authenticated using refresh token.


### init the job, processing and collection option depending on provider and task

In [3]:
job_options = get_job_options(provider=backend, task='data_extraction')
collection_options = get_collection_options(provider=backend)
# reset S1, DEM and Agera5 to only get Sentinel2 data cube
collection_options.update(METEO_collection=None, S1_collection=None, DEM_collection=None)
processing_options = get_advanced_dataextraction_options(provider=backend)

In [4]:
# just print for an overview
print(f'job_options: {job_options}')
print(f'collection_options: {collection_options}')
print(f'processing_options: {processing_options}')

job_options: {'driver-memory': '4G', 'driver-memoryOverhead': '4G', 'driver-cores': '2', 'executor-memory': '3G', 'executor-memoryOverhead': '2G', 'executor-cores': '2', 'max-executors': '50', 'soft-errors': 'true'}
collection_options: {'S2_collection': 'SENTINEL2_L2A', 'METEO_collection': None, 'S1_collection': None, 'DEM_collection': None}
processing_options: {'provider': 'development', 's1_orbitdirection': 'DESCENDING', 'target_crs': 3035, 'resolution': 10.0, 'time_interpolation': False, 'ts_interval': 'dekad', 'SLC_masking_algo': 'mask_scl_dilation'}


### specify the space and time context

In [5]:
# the time context is given by start and end date
start = '2020-01-01'
end = '2021-01-01'   # the end is always exclusive

In [6]:
# the space context is defined as a bounding box dictionary with south,west,north,east and crs
# we take as example the AOI of Daniele's test for habitat mapping
AOI = {'east': 601000, 'south': 5699000, 'west': 600000, 'north': 5700000, 'crs': 'EPSG:32631'}

In [7]:
# create area info for AOI
from shapely.geometry import box
import geopandas as gpd
df = gpd.GeoDataFrame({"id":1,"geometry":[box(AOI['west'], AOI['south'], AOI['east'], AOI['north'])]})
df.crs = AOI['crs']
print(f'area of AOI in km2: {df.iloc[0].geometry.area/ 10**6}')

area of AOI in km2: 1.0


### run over the AOI and process

In [8]:
test_num = 6

# define the output folder root
out_root = os.path.normpath(r'C:\Users\BUCHHORM\OneDrive - VITO\Documents\Project_work\PEOPLE-EA\habitat_mapping\VI_generation')
out_root = os.path.join(out_root, f'test_v{str(test_num)}')
os.makedirs(out_root, exist_ok=True)

In [ ]:
# specify the dictionary to specify the VIs and scaling ranges of input and output
# NDMI == NDWI after Gao; NDWI == NDWI after McFeeter
S2_vi_dict = {
    "collection": {
        "input_range": [0,10000],
        "output_range": [0,1]
    },
    "indices": {
        "NDVI": {
            "input_range": [-1,1],
            "output_range": [-10000,10000]
        },
        "ATSAVI": {
            "input_range": [-1,1],
            "output_range": [-10000,10000]
        },
        "AVI": {
            "input_range": [0,1],
            "output_range": [0,10000]
        },
        "CIRE": {
            "input_range": [-1,1],
            "output_range": [-10000,10000]
        },
        "EVI": {
            "input_range": [-10,10],
            "output_range": [-20000,20000]
        },
        "NIRv": {
            "input_range": [-1,1],
            "output_range": [-10000,10000]
        },
        "SIPI": {
            "input_range": [-10,10],
            "output_range": [-20000,20000]
        },
        "NDMI": {
            "input_range": [-1,1],
            "output_range": [-10000,10000]
        },
        "NDWI": {
            "input_range": [-1,1],
            "output_range": [-10000,10000]
        },
        "NBRplus": {
            "input_range": [-1,1],
            "output_range": [-10000,10000]
        },
        "BAIS2": {
            "input_range": [-1,6],
            "output_range": [-1000,6000]
        },
        "kNDVI": {
            "input_range": [0,1],
            "output_range": [0,10000]
        },
        "BLFEI": {
            "input_range": [-1,1],
            "output_range": [-10000,10000]
        },
        "MNDWI": {
            "input_range": [-1,1],
            "output_range": [-10000,10000]
        },
        "NDVIMNDWI": {
            "input_range": [-2,2],
            "output_range": [-20000,20000]
        },
        "S2WI": {
            "input_range": [-1,1],
            "output_range": [-10000,10000]
        },
        "SAVI": {
            "input_range": [-1,1],
            "output_range": [-10000,10000]
        },
        "S2REP": {
            "input_range": [700,800],
            "output_range": [7000,8000]
        },
        "IRECI": {
            "input_range": [0,10],
            "output_range": [0,10000]
        },
        "NDGI": {
            "input_range": [-1,1],
            "output_range": [-10000,10000]
        },
    }
}

In [9]:
### due to issue with the awesome package implementation in openEo NO VI's with constants can be used
## also the band definition for Sentinel2 is not correct in the openEO implementation (RE4 should be N2)
S2_vi_dict = {
    "collection": {
        "input_range": [0,10000],
        "output_range": [0,1]
    },
    "indices": {
        "NDVI": {
            "input_range": [-1,1],
            "output_range": [-10000,10000]
        },
        "AVI": {
            "input_range": [0,1],
            "output_range": [0,10000]
        },
        "CIRE": {
            "input_range": [-1,1],
            "output_range": [-10000,10000]
        },
        "NIRv": {
            "input_range": [-1,1],
            "output_range": [-10000,10000]
        },
        "NDMI": {
            "input_range": [-1,1],
            "output_range": [-10000,10000]
        },
        "NDWI": {
            "input_range": [-1,1],
            "output_range": [-10000,10000]
        },
        "BLFEI": {
            "input_range": [-1,1],
            "output_range": [-10000,10000]
        },
        "MNDWI": {
            "input_range": [-1,1],
            "output_range": [-10000,10000]
        },
        "NDVIMNDWI": {
            "input_range": [-2,2],
            "output_range": [-20000,20000]
        },
        "S2WI": {
            "input_range": [-1,1],
            "output_range": [-10000,10000]
        },
        "S2REP": {
            "input_range": [700,800],
            "output_range": [7000,8000]
        },
        "IRECI": {
            "input_range": [0,10],
            "output_range": [0,10000]
        },
    }
}

In [12]:
print(f'**** processing AOI')
# define the job to get input data --> here only Sentinel-2 datacube
S2_ts_cube = ts_datacube_extraction(connection,
                                   AOI,
                                   start,
                                   end,
                                   **collection_options,
                                   **processing_options)
# rescale S2 data cube into Int16
S2_ts_cube = S2_ts_cube.linear_scale_range(-32767, 32767, -32767, 32767)


# add the VI generation to the cube (later that comes in a warper function in openeo folder of extentmapping package])
# warper should automatically reduce to S2 cube by altering the collection options for ts_datacube_extraction

S2_vi_cube = compute_and_rescale_indices(datacube=S2_ts_cube, index_dict=S2_vi_dict, append=False)
# force Int16
S2_vi_cube = S2_vi_cube.linear_scale_range(-32767, 32767, -32767, 32767)


S2_ts_cube_extented = S2_ts_cube.merge_cubes(S2_vi_cube)


# run it
try:
    job = S2_ts_cube_extented.execute_batch(title=f'VI generation for AOI - test {str(test_num)}',
                                  description='Generation of VI from S2 via OpenEO',
                                  out_format='NetCDF',
                                  job_options=job_options)

    # Get the results and save to output file
    results = job.get_results()

    outputfile = os.path.join(out_root, f'S2_ref-and-VI_timeseries_openeo_AOI_test-v{str(test_num)}.nc')
    results.download_file(outputfile)
    # get metadata and job info
    with open(outputfile.replace('.nc','.json'), "w") as outfile:
        json.dump(results.get_metadata(),outfile)
    with open(outputfile.replace('.nc','-job.json'), "w") as outfile:
        json.dump(job.describe_job(),outfile)

except Exception as e:
    print(e)

**** processing AOI
0:00:00 Job 'j-2311120a884e4280b72f207d00e4d9a5': send 'start'
0:00:19 Job 'j-2311120a884e4280b72f207d00e4d9a5': queued (progress N/A)
0:00:25 Job 'j-2311120a884e4280b72f207d00e4d9a5': queued (progress N/A)
0:00:31 Job 'j-2311120a884e4280b72f207d00e4d9a5': queued (progress N/A)
0:00:39 Job 'j-2311120a884e4280b72f207d00e4d9a5': queued (progress N/A)
0:00:49 Job 'j-2311120a884e4280b72f207d00e4d9a5': queued (progress N/A)
0:01:02 Job 'j-2311120a884e4280b72f207d00e4d9a5': queued (progress N/A)
0:01:17 Job 'j-2311120a884e4280b72f207d00e4d9a5': queued (progress N/A)
0:01:37 Job 'j-2311120a884e4280b72f207d00e4d9a5': queued (progress N/A)
0:02:01 Job 'j-2311120a884e4280b72f207d00e4d9a5': queued (progress N/A)
0:02:31 Job 'j-2311120a884e4280b72f207d00e4d9a5': queued (progress N/A)
0:03:08 Job 'j-2311120a884e4280b72f207d00e4d9a5': queued (progress N/A)
0:03:55 Job 'j-2311120a884e4280b72f207d00e4d9a5': running (progress N/A)
0:04:53 Job 'j-2311120a884e4280b72f207d00e4d9a5': ru

In [ ]:
# load data and check the last processed file
import xarray
ts = xarray.load_dataset(outputfile)
ts